In [1]:
import os,glob,shutil,subprocess,sys,pickle
import itertools as itt
import pandas as pd
import numpy as np
import toml

In [2]:
#adjust this to your needs 
#the current form was written for IBM Spectrum LSF Standard 10.1.0.12
def HPC_wrapper(command : str, queue : str = "verylong" , memory : int = 20, cores : int = 1) -> str:
    first_part =   f"bsub -R \"rusage[mem={memory}G] span[hosts=1] affinity[core({cores})]\" -q {queue} -o {WORKING_DIR}logs \" "
    last_part = " \" \n"
    
    return first_part + command + last_part


In [4]:
#directory with folder structure produced by setup.sh
WORKING_DIR = "/b06x-isi/b062/a-c/Braun_CRISPR_2D/"

In [ ]:
'''
below is the minimum structure required for the pipeline

WORKING_DIR
├── dLFC_analysis
├── GEMINI_input
├── processed_sequencing_data
├── raw_sequencing_data
├── Snakefile
└── scripts
'''

In [4]:
os.chdir(WORKING_DIR)

The following assumes you already ran the Snakemake pipeline and obtained the files with the counts.

# Collect the count results 

Collect and aggreagte the count results file produced by snakemake pipeline from the processed_sequencing_data directory

In [9]:
#list with names of folder you wish to collect the count results from
subfolders_with_countresults = ["RPE_2Dscreen","T98G_2DScreen","X204SC20113232-Z01-F010","X204SC20113232-Z01-F007"]
celllines = ["RPE","T98G","PATU","HEK"]

In [ ]:
#run the aggregate_counts for each subfolder
for folder in subfolders_with_countresults:
    gemini_output_folder = WORKING_DIR+"GEMINI_input/"+folder+"/"
    if not os.path.exists(gemini_output_folder):
        try:
            os.makedirs(gemini_output_folder)
        except Exception as e:
            print(e)
            break
          
    #run once for raw counts
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/aggregate_counts.py", #call script
                    "--directory",folder, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--outputdir",gemini_output_folder, #path where results will be written to
                   ])
    
    #run once for counts as CPM
    '''
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/aggregate_counts.py", #call script
                    "--directory",folder, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--outputdir",gemini_output_folder, #path where results will be written to
                    "--normalize",
                   ])
    '''

In [ ]:
#merges the results from the previous steps and removes redundant files
for folder,cellline in zip(subfolders_with_countresults,celllines):
    counts_folder = WORKING_DIR+"GEMINI_input/"+folder+"/"
    
    #aggregate raw counts
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/merge_counts.py", #call script
                    "--cellline",cellline, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--counts_dir",counts_folder, #path where results will be written to
                   ])
    
    #aggregate normalized counts
    '''
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/merge_counts.py", #call script
                    "--cellline",cellline, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--counts_dir",counts_folder, #path where results will be written to
                    "--normalize",
                   ])
    '''

In [ ]:
#if you need to rename the columns to something more usefull do it now
files_raw = ["/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/RPE_2Dscreen/merged_counts_RPE_raw.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/T98G_2DScreen/merged_counts_T98G_raw.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/X204SC20113232-Z01-F010/merged_counts_PATU_raw.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/X204SC20113232-Z01-F007/merged_counts_HEK_raw.csv"]

files_norm = ["/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/RPE_2Dscreen/merged_counts_RPE_norm.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/T98G_2DScreen/merged_counts_T98G_norm.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/X204SC20113232-Z01-F010/merged_counts_PATU_norm.csv",
 "/b06x-isi/b062/a-c/Braun_CRISPR_2D/GEMINI_input/X204SC20113232-Z01-F007/merged_counts_HEK_norm.csv"]

rename_dicts = [{"RPE_FF6":"RPETP3.REP2", "RPE_AA1":"RPETP1.REP1", "RPE_EE5":"RPETP2.REP2", "RPE_DD4":"RPETP1.REP2", "RPE_CC3":"RPETP3.REP1", "RPE_BB2":"RPETP2.REP1"},
               {"s0713_BB2_2_5":"T98GTP2.REP1","s0713_DD4_1_7":"T98GTP1.REP2","s0713_AA1_1_4":"T98GTP1.REP1","s0713_CC3_3_6":"T98GTP3.REP1","s0713_FF6_3_5":"T98GTP3.REP2","s0713_EE5_2_4":"T98GTP2.REP2"},
               {"PA_TU8988_D4":"PATUTP1.REP2","PA_TU8988_F6":"PATUTP3.REP2","PA_TU8988_B2":"PATUTP2.REP1","PA_TU8988_E5":"PATUTP2.REP2","PA_TU8988_A1":"PATUTP1.REP1","PA_TU8988_C3":"PATUTP3.REP1"},
               {"C3_3_6":"HEKTP2.REP1","E5_2_4":"HEKTP3.REP2","F6_3_5":"HEKTP2.REP2","D4_1_7":"HEKTP1.REP2","A1_1_4":"HEKTP1.REP1","B2_2_5":"HEKTP3.REP1"}]


In [ ]:
#actually renaming columns
for f,d in zip(files_raw,rename_dicts):
    df = pd.read_csv(f,sep="\t",index_col=0)
    df = df.rename(d,axis=1)
    df.to_csv(f,sep="\t")
    
for f,d in zip(files_norm,rename_dicts):
    df = pd.read_csv(f,sep="\t",index_col=0)
    df = df.rename(d,axis=1)
    df.to_csv(f,sep="\t")

In [ ]:
#copy merged count results into dLFC folder
for src in glob.glob(WORKING_DIR+"GEMINI_input/*/merged_counts_*.csv"):
    name = src.split("/")[-1]
    dst = WORKING_DIR+"dLFC_analysis/"+name
    try:
        shutil.copyfile(src,dst)
    except Exception as e:
        print(e)

In [ ]:
#copy toml files into dLFC folder
for src in glob.glob(WORKING_DIR+"GEMINI_input/*/config_*.toml"):
    name = src.split("/")[-1]
    dst = WORKING_DIR+"dLFC_analysis/"+name
    try:
        shutil.copyfile(src,dst)
    except Exception as e:
        print(e)

# Run GEMINI

In [ ]:
path_anno_file = WORKING_DIR + "metainfo/guide_anno.csv"
path_sample_anno = WORKING_DIR + "metainfo/sample_replicate_anno.csv"
path_exclusion_file = WORKING_DIR + "metainfo/combinations_affected_by_TTTTgate.tsv"

with open(WORKING_DIR + "submission_script.sh","w+") as file:
    file.write("module load R/4.1.0\n")
    file.write("\n")
    for folder,cellline in zip(subfolders_with_countresults,celllines):
        if cellline != "PATU":
            continue
        path_counts_file = WORKING_DIR + f"GEMINI_input/{folder}/merged_counts_{cellline}_raw.csv"
        path_config_file = WORKING_DIR + f"GEMINI_input/{folder}/config_{cellline}.toml"
    
        command = " ".join(["Rscript",WORKING_DIR+"scripts/gemini_single.R", #call script
                            path_anno_file,
                            path_sample_anno,
                            path_counts_file, 
                            path_exclusion_file,
                            str(4),
                            WORKING_DIR,
                            path_config_file
                           ])
    
        file.write(HPC_wrapper(command = command,queue= "medium" , memory= 10, cores= 6))

# Run dLFC analysis

In [33]:
#preparing LFC values from raw count values
for path_count_file in glob.glob(WORKING_DIR+"dLFC_analysis/merged_counts_*_raw.csv"):
    cellline = path_count_file.split("/")[-1].split("_")[2]

    path_config_file = WORKING_DIR+"dLFC_analysis/config_{}.toml".format(cellline)
    
    output_dir = WORKING_DIR + "dLFC_analysis/"
    path_anno_file = WORKING_DIR + "metainfo/guide_anno.csv"
    path_exclusion_file = WORKING_DIR + "metainfo/combinations_affected_by_TTTTgate.tsv"
    path_sample_anno = WORKING_DIR + "metainfo/sample_replicate_anno.csv"
    
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/prepare_raw_LFC.py", #call script
                    "--output_dir",output_dir, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--path_count_file",path_count_file, #path to counts files
                    "--path_config_file",path_config_file,
                    "--path_anno_file",path_anno_file,
                    "--path_exclusion_file",path_exclusion_file,
                    "--path_sample_anno",path_sample_anno,
                   ])


In [46]:
#processing the LFC values to obtain expected and observed values
#this step is recommended to be run in some sort of HPC cluster
comparisons = ["12","13","23"]

with open(WORKING_DIR + "submission_script.sh","w+") as file:
    file.write(f"cd {WORKING_DIR}dLFC_analysis \n")
    file.write("module load anaconda3/2021.05 \n")
    file.write("source activate cas13_pipeline \n")
    
    for cellline in celllines:
        for comp in comparisons:
            lfc_file = WORKING_DIR+"dLFC_analysis/raw_LFCZ_{}_{}.tsv".format(cellline,comp)
    
            output_dir = WORKING_DIR + "dLFC_analysis/"
        
            command = " ".join(["python",WORKING_DIR+"scripts/process_raw_LFC.py", #call script
                    "--output_dir",output_dir, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--cellline",cellline, #path to counts files
                    "--comparison",comp,
                    "--lfc_file",lfc_file,
                   ])
    
            file.write(HPC_wrapper(command))


#however if you are comfortable using it in the context of this notebook use the code below
'''
for path_count_file in glob.glob(WORKING_DIR+"dLFC_analysis/merged_counts_*.csv"):
    cellline = path_count_file.split("/")[-1].split("_")[2]

    path_config_file = WORKING_DIR+"dLFC_analysis/config_{}.toml".format(cellline)
    
    output_dir = WORKING_DIR + "dLFC_analysis/"
    path_anno_file = WORKING_DIR + "metainfo/guide_anno.csv"
    path_exclusion_file = WORKING_DIR + "metainfo/combinations_affected_by_TTTTgate.tsv"
    path_sample_anno = WORKING_DIR + "metainfo/sample_replicate_anno.csv"
    
    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/process_raw_LFC.py", #call script
                    "--output_dir",output_dir, #setting the subfolder count results will be extracted from
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--cellline",path_count_file, #path to counts files
                    "--comparison",path_config_file,
                    "--lfc_file",path_anno_file,
                   ])
'''

'\nfor path_count_file in glob.glob(WORKING_DIR+"dLFC_analysis/merged_counts_*.csv"):\n    cellline = path_count_file.split("/")[-1].split("_")[2]\n\n    path_config_file = WORKING_DIR+"dLFC_analysis/config_{}.toml".format(cellline)\n    \n    output_dir = WORKING_DIR + "dLFC_analysis/"\n    path_anno_file = WORKING_DIR + "metainfo/guide_anno.csv"\n    path_exclusion_file = WORKING_DIR + "metainfo/combinations_affected_by_TTTTgate.tsv"\n    path_sample_anno = WORKING_DIR + "metainfo/sample_replicate_anno.csv"\n    \n    subprocess.run([f"{sys.executable}",WORKING_DIR+"scripts/process_raw_LFC.py", #call script\n                    "--output_dir",output_dir, #setting the subfolder count results will be extracted from\n                    "--workingdir",WORKING_DIR, #passing working dir to script\n                    "--cellline",path_count_file, #path to counts files\n                    "--comparison",path_config_file,\n                    "--lfc_file",path_anno_file,\n                 

In [29]:
#run dLFC rank test
all_cellline_TP_combis = [["HEK","12"],["HEK","13"],["HEK","23"],
         ["T98G","12"],["T98G","13"],["T98G","23"],
         ["PATU","12"],["PATU","13"],["PATU","23"],
         ["RPE","12"],["RPE","13"],["RPE","23"]]

with open(WORKING_DIR + "submission_script.sh","w+") as file:
    file.write(f"cd {WORKING_DIR}dLFC_analysis \n")
    file.write("module load anaconda3/2021.05 \n")
    file.write("source activate cas13_pipeline \n")
    
    for cellline,comparison in all_cellline_TP_combis:
        
        command = " ".join(["python",WORKING_DIR+"scripts/dLFC_ranktest.py", #call script
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--cellline",cellline, #path to counts files
                    "--comparison",comparison, #comparison of timepoints
                   ])
    
        file.write(HPC_wrapper(command))

# T-test

In [35]:
#run t-test 
all_cellline_TP_combis = [["HEK","12"],["HEK","13"],["HEK","23"],
         ["T98G","12"],["T98G","13"],["T98G","23"],
         ["PATU","12"],["PATU","13"],["PATU","23"],
         ["RPE","12"],["RPE","13"],["RPE","23"]]
exclusionlist = WORKING_DIR + "metainfo/common_essential_genes.tsv"

with open(WORKING_DIR + "submission_script.sh","w+") as file:
    file.write(f"cd {WORKING_DIR}dLFC_analysis \n")
    file.write("module load anaconda3/2021.05 \n")
    file.write("source activate cas13_pipeline \n")
    
    for cellline,comparison in all_cellline_TP_combis:
        
        command = " ".join(["python",WORKING_DIR+"scripts/ttest.py", #call script
                    "--workingdir",WORKING_DIR, #passing working dir to script
                    "--cellline",cellline, #path to counts files
                    "--comparison",comparison, #comparison of timepoints
                    #"--exclusionlist",exclusionlist,
                   ])
    
        file.write(HPC_wrapper(command,"medium"))